In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\MlOps\\End-to-end-Machine-Learning-Project-with-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\MlOps\\End-to-end-Machine-Learning-Project-with-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int
    max_features: str



In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            min_samples_split = params.min_samples_split,
            min_samples_leaf = params.min_samples_leaf,
            max_features = params.max_features,
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from src.mlProject import logger
from sklearn.ensemble import RandomForestRegressor
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path, index_col=None)
        test_data = pd.read_csv(self.config.test_data_path, index_col=None)


        X_train = train_data.drop(['price'], axis=1)
        X_test = test_data.drop(['price'], axis=1)
        y_train = train_data[['price']]
        y_test = test_data[['price']]


        lr = RandomForestRegressor(n_estimators=self.config.n_estimators, min_samples_split=self.config.min_samples_split, 
                                   min_samples_leaf=self.config.min_samples_leaf, max_features=self.config.max_features, random_state=42)
        lr.fit(X_train, y_train)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))



In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-12-18 22:32:19,951: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-18 22:32:19,963: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-18 22:32:19,965: INFO: common: created directory at: artifacts]
[2023-12-18 22:32:19,969: INFO: common: created directory at: artifacts/model_trainer]


d:\Program Files\Anaconda3\envs\python_3\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
